# 1 Data import

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import scipy.stats as sct
import pandas_ta as pta
from sklearn.ensemble import RandomForestRegressor
import random
from datetime import timedelta

In [ ]:
# Dataframe with the highest granularity of prices
df_prices = pd.read_csv(r'C:\Users\luigi\OneDrive\2 University\4 ULB\0 THESIS\DATA\DF_big_processed.csv',index_col=['timestamp'],parse_dates=['timestamp','datetime'])

# Dataframe with the data used to train the RF (in this case, the 15s dataset)
df = pd.read_csv(r'C:\Users\luigi\OneDrive\2 University\4 ULB\0 THESIS\DATA\df_15s_processed.csv',parse_dates=['timestamp','datetime'])
df = df.set_index('timestamp')
df['timestamp'] = df.index


# Dataset of the trading parameters from N-SIDE
df_trading = pd.read_csv(r'C:\Users\luigi\OneDrive\2 University\4 ULB\0 THESIS\N-SIDE trading strategy\output_trading_strategy.csv',parse_dates=['open_position_time','datetime','target_time','OB_closure_timestamp','creation_time'])

# optimal s identified when training the RF models (hyperparameter for the Technical indicators)
s_opt = pd.read_csv(r'C:\Users\luigi\OneDrive\2 University\4 ULB\0 THESIS\DATA\CANTIERE_NAVALE\s_opt.csv')

In [ ]:
df_trading = df_trading[june_mask]
df_trading.index = np.array(range(0,595))

# 2 Analysis of the trading dataset from N-SIDE

- Datetime is the column of the latest moment at which the position can be closed
- target_time is the column of the moment when the position is closed

## 2.1 plot of a trading operation

In [ ]:
# We select the trading operation (any)
i = 46
mask = df_trading['Unnamed: 0'] == i


# We define some plotting parameters
selected_month = int(df_trading.open_position_time[mask].dt.month)
selected_day = int(df_trading.open_position_time[mask].dt.day)
selected_hour = int(df_trading.open_position_time[mask].dt.hour)
selected_minute = int(df_trading.open_position_time[mask].dt.minute)

selected_time = [selected_month, selected_day, selected_hour, selected_minute]


# The interval we will plot are the day of the position and the day after
plot_mask = (df_prices.month == selected_month) & ((df_prices.day == selected_day) | (df_prices.day == (selected_day + 1)))


# We extract opening time and price, closing time and price, deadline
open_time = df_trading.open_position_time[mask][i]
open_price = float(df_trading.price_at_open_time[mask])

target_time = df_trading.target_time[mask][i]
closing_price = float(df_trading.closing_price[mask])
deadline = df_trading.datetime[mask][i]
price_diff = abs(closing_price-open_price)

# We want to have the green line above the red when we make a profit.
# If we have a long position (buy then sell) we will have the green line for the sell price and the red line for the buy price.
# If we have a short position (sell then buy) we will have the green line for the buy price and the red line for the sell price.
      
if (int(df_trading.opening_type[mask] == 'buy')):
    plt.figure(figsize = (20,10))
    
    plt.plot(df_prices.price[plot_mask])
    plt.axvline(x=(open_time), c='r')
    plt.axhline(y=open_price, color='r', linestyle='--', label='opening price (buy)')

    plt.axvline(x=target_time, c='g')
    plt.axhline(y=closing_price, color='g', linestyle='--', label='closing price (sell)')
                      
    plt.axvline(x=deadline, c='y')
                      
    plt.ylim(closing_price-price_diff-10,closing_price+price_diff+10)
    
    if (open_price-closing_price < 0):
        
        plt.fill_between(df_prices.index[plot_mask],
                         closing_price,
                        open_price, 
                        color='g', alpha=.15)
    else:
        plt.fill_between(df_prices.index[plot_mask],
                         closing_price,
                        open_price, 
                        color='r', alpha=.15)
    plt.title('Long trading position', size=30)
    plt.legend(prop={'size': 20})
    plt.ylabel('€', size=20)
    plt.show()

else:
    plt.figure(figsize = (20,10))                    
    plt.plot(df_prices.price[plot_mask])
    
    plt.axvline(x=(open_time), c='g')
    plt.axhline(y=open_price, color='g', linestyle='--', label='opening price (sell)')

    plt.axvline(x=target_time, c='r')
    plt.axhline(y=closing_price, color='r', linestyle='--', label='closing price (buy)')
                      
    plt.axvline(x=deadline, c='y')
                      
    
    plt.ylim(closing_price-price_diff-10,closing_price+price_diff+10)
    
    if (open_price-closing_price < 0):
        
        plt.fill_between(df_prices.index[plot_mask],
                         closing_price,
                        open_price, 
                        color='r', alpha=.15)
    else:
        plt.fill_between(df_prices.index[plot_mask],
                         closing_price,
                        open_price, 
                        color='g', alpha=.15)
                      
    plt.legend(prop={'size': 20})
    plt.title('Short trading position', size=30)
    plt.ylabel('€', size=20)
    plt.show()
    
print('The operation is: ',df_trading.opening_type[mask][i])
print('Opening price: ',open_price,' at ',open_time,' , closing price: ', closing_price,' at ', target_time)
print('The profit corresponds to: ', float(df_trading.savings[mask]),' €')
print('The position was closed due to: ',df_trading.closing_mode[mask][i])

# 2.2 Plot of the output of the forecasting tool

In [ ]:
# We plot it on the same timeframe of the previous operation
trading_plot_mask = (df_trading.open_position_time.dt.month == selected_month) & ((df_trading.open_position_time.dt.day == selected_day) | (df_trading.open_position_time.dt.day == (selected_day + 1)))

plt.figure(figsize = (20,10)) 

plt.axhline(y=0, color='black', linestyle='--')
plt.axhline(y=1, color='darkgreen', linestyle='--')
plt.axhline(y=2, color='seagreen', linestyle='--' )
plt.axhline(y=3, color='limegreen', linestyle='--')
plt.axhline(y=4, color='lime', linestyle='--')

plt.axhline(y=-1, color='darksalmon', linestyle='--')
plt.axhline(y=-2, color='coral', linestyle='--' )
plt.axhline(y=-3, color='darkorange', linestyle='--')
plt.axhline(y=-4, color='r', linestyle='--')

plt.plot(df_trading.open_position_charge[trading_plot_mask], 'b', label='output')
plt.title('Output of the forecasting tool', size=30)
plt.legend(prop={'size': 20})

## 2.3 Analysis of the testing parameters
Only the days from the 11th of june will be considered for testing purposes

In [ ]:
# we apply a mask to select only the period of interest
june_mask = (df_trading.datetime.dt.month == 6) & (df_trading.datetime.dt.day > 10)
number_operations = df_trading.datetime[june_mask].count()
total_profit = np.sum(df_trading.savings[june_mask])
profit = df_trading.savings[june_mask].values
print('The number of operations under analysis is: ',number_operations)
print('The total profit realized with the positions opened by N-SIDE is: ',np.round(total_profit,2),' €')

In [ ]:
basic_stats = sct.describe(profit,nan_policy='omit')
print(basic_stats)

## 2.4 Specific analysis of the savings

In [ ]:
N = len(profit)
h = sct.iqr(profit, nan_policy='omit') * 2 * N**(-1/3)
n_bins = int(np.floor((np.nanmax(profit)-np.nanmin(profit))/h))

plt.figure(figsize = (20,10)) 
plt.hist(df_trading.savings[june_mask], bins = n_bins,alpha=0.8, label='profit')
plt.axvline(x=basic_stats.mean,c='r', label='mean')
plt.title('Histogram of the profits of the current trading strategy', size=30)
plt.xlabel('€', size=20)
plt.ylabel('frequency', size=20)
plt.legend(prop={'size': 20})

plt.show()
print('The mean profit is: ',round(basic_stats.mean,2),' €')

# 3 Trading strategy with ML

In [ ]:
df_trading = df_trading[june_mask]
df_trading.index = np.array(range(0,595))

## 3.1 [DEPRECATED, SKIP] Model re-trained at every iteration

In [ ]:
def rf_hourly_forecast(open_time,df_15s,deadline,df_prices,s_opt):
    
    starting_month = open_time.month
    starting_day = open_time.day
    starting_hour = open_time.hour
    starting_minute = open_time.minute


    ending_month = deadline.month
    ending_day = deadline.day
    ending_hour = deadline.hour
    ending_minute = deadline.minute
    
    if ending_day == starting_day:
        hours_forecast = range(starting_hour,(ending_hour+1))
    else:
        hours_forecast = np.concatenate([np.array(range(starting_hour,24)),np.array(range(0,ending_hour+1))])

    predictions = []
    real_values = []
    esse = []
    timeframe = []
    df_predictions = pd.DataFrame()
    for hour in hours_forecast:
        s = s_opt.RMSE[hour]
        esse.append(s)

        selected_hour = hour
        
        if ((ending_day != starting_day) & (hour >= 12)):
            current_day = starting_day
        elif ((ending_day != starting_day) & (hour < 12)):
            current_day = ending_day

        else:
            current_day = starting_day
                
                
        ### 1  CALCULATION OF TI
        sma =df.price.rolling(window=s).mean()
        mom = df.price.diff(s)
        rsi = pta.rsi(df.price, length = s)
        mstd =df.price.rolling(window=s).std()
        ema_1 = df.price.ewm(span=7, adjust=False).mean()
        ema_2 = df.price.ewm(span=14, adjust=False).mean()
        macd = ema_1 - ema_2
        BB_upper = sma + 2*mstd
        BB_lower = sma - 2*mstd
        ppcma = df.price.pct_change(periods=s)
        mad = (df.price-ppcma)/ppcma
        rolling_max = df.price.rolling(window=s).max()
        rolling_min = df.price.rolling(window=s).min()
        pr = ((rolling_max-df.price)/(rolling_max-rolling_min))*100
        A = rolling_max - rolling_min
        B = np.abs(rolling_max - df.price)
        C = np.abs(rolling_min - df.price)
        ABC=pd.DataFrame(A)
        ABC['B'] = B
        ABC['C'] = C
        MAX = ABC.max(axis=1)
        atr = MAX.rolling(window=s).mean()

        ### 2 CREATION OF A DATAFRAME OF TI

        features=pd.DataFrame(df.price)
        features['sma'] = sma
        features['mom'] = mom
        features['rsi'] = rsi
        features['mstd'] = mstd
        features['macd'] = macd
        features['BB_upper'] = BB_upper
        features['BB_lower'] = BB_lower
        features['ppcma'] = ppcma
        features['mad'] = mad
        features['pr'] = pr
        features['atr'] = atr
        features['month'] = df.month
        features['day'] = df.day
        features['weekday'] = df.timestamp.dt.weekday
        features['hour'] = df.hour
        features['minute'] = df.minute

        label = list(df.price[1:])
        label.append(0)
        features['label'] = label

        del features['price']

        ### 3 CREATING HOURLY FEATURES

        features_hour = features[features.hour == selected_hour]
        features_hour.replace([np.inf, -np.inf], np.nan, inplace=True)
        features_hour.fillna(-999, inplace = True)

        ### 4 CREATING A TRAINING AND A TESTING SET

        train_features = features_hour[(features_hour.month == 5) | ((features_hour.month == 6) & (features_hour.day <= 10)) ]
        train_labels = features_hour.label[(features_hour.month == 5) | ((features_hour.month == 6) & (features_hour.day <= 10))]

        test_features = features_hour[(features_hour.month == 6)
                                  & (features_hour.day == current_day)]

        test_labels = features_hour.label[(features_hour.month == 6)
                                  & (features_hour.day == current_day)]

        del train_features['label']
        del test_features['label']

        rf = RandomForestRegressor(n_estimators = 100, random_state = 42)
        rf.fit(train_features, train_labels)

        ### 6 ESTIMATION OF THE PREDICTIONS AND OF THE ERRORS
        predictions = rf.predict(test_features)
        real_values = test_labels.values
        #predictions= np.concatenate([predictions, rf.predict(test_features)])
        #real_values = np.concatenate([real_values, test_labels.values])
        
        df_predictions_temp = pd.DataFrame(test_labels.index)
        df_predictions_temp['predictions'] = predictions
        df_predictions_temp['real_values'] = real_values
        
        df_predictions = pd.concat([df_predictions,df_predictions_temp])
        

    plt.figure(figsize = (20,10))
    plt.plot(df_predictions.timestamp,df_predictions.predictions,'r', label='predictions')
    plt.plot(df_predictions.timestamp,df_predictions.real_values,'b', label='real values')
    plt.legend(prop={'size': 20})
    plt.title('Predictions of the RF models', size=30)
    plt.ylabel('€', size=20)
    plt.show()
    
    n = len(df_predictions.predictions)
    RMSE = np.sqrt(1/n*np.sum((df_predictions.predictions
                               -df_predictions.real_values)**2))
    print(RMSE)
    df_predictions.index = np.array(range(0,len(df_predictions.predictions)))
    return(df_predictions)
    

In [ ]:
i = 0
mask = df_trading.index == i


# We define some plotting parameters
selected_month = int(df_trading.open_position_time[mask].dt.month)
selected_day = int(df_trading.open_position_time[mask].dt.day)
selected_hour = int(df_trading.open_position_time[mask].dt.hour)
selected_minute = int(df_trading.open_position_time[mask].dt.minute)

selected_time = [selected_month, selected_day, selected_hour, selected_minute]

plot_mask = (df_prices.month == selected_month) & ((df_prices.day == selected_day) | (df_prices.day == (selected_day + 1)))


open_time = df_trading.open_position_time[mask][i]
open_price = float(df_trading.price_at_open_time[mask])

target_time = df_trading.target_time[mask][i]
closing_price = float(df_trading.closing_price[mask])
deadline = df_trading.datetime[mask][i]
price_diff = abs(closing_price-open_price)

# We want to have the green line above the red when we make a profit.
# If we have a long position (buy then sell) we will have the green line for the sell price and the red line for the buy price.
# If we have a short position (sell then buy) we will have the green line for the buy price and the red line for the sell price.
      
if (int(df_trading.opening_type[mask] == 'buy')):
    plt.figure(figsize = (20,10))
    
    plt.plot(df_prices.price[plot_mask])
    plt.axvline(x=(open_time), c='r')
    plt.axhline(y=open_price, color='r', linestyle='--', label='opening price (buy)')

    plt.axvline(x=target_time, c='g')
    plt.axhline(y=closing_price, color='g', linestyle='--', label='closing price (sell)')
                      
    plt.axvline(x=deadline, c='y')
                      
    plt.ylim(closing_price-price_diff-10,closing_price+price_diff+10)
    
    if (open_price-closing_price < 0):
        
        plt.fill_between(df_prices.index[plot_mask],
                         closing_price,
                        open_price, 
                        color='g', alpha=.15)
    else:
        plt.fill_between(df_prices.index[plot_mask],
                         closing_price,
                        open_price, 
                        color='r', alpha=.15)
    plt.title('Long trading position', size=30)
    plt.legend(prop={'size': 20})
    plt.ylabel('€', size=20)
    plt.show()

else:
    plt.figure(figsize = (20,10))                    
    plt.plot(df_prices.price[plot_mask])
    
    plt.axvline(x=(open_time), c='g')
    plt.axhline(y=open_price, color='g', linestyle='--', label='opening price (sell)')

    plt.axvline(x=target_time, c='r')
    plt.axhline(y=closing_price, color='r', linestyle='--', label='closing price (buy)')
                      
    plt.axvline(x=deadline, c='y')
                      
    
    plt.ylim(closing_price-price_diff-10,closing_price+price_diff+10)
    
    if (open_price-closing_price < 0):
        
        plt.fill_between(df_prices.index[plot_mask],
                         closing_price,
                        open_price, 
                        color='r', alpha=.15)
    else:
        plt.fill_between(df_prices.index[plot_mask],
                         closing_price,
                        open_price, 
                        color='g', alpha=.15)
                      
    plt.legend(prop={'size': 20})
    plt.title('Short trading position', size=30)
    plt.ylabel('€', size=20)
    plt.show()
    
print('The operation is: ',df_trading.opening_type[mask][i])
print('Opening price: ',open_price,' at ',open_time,' , closing price: ', closing_price,' at ', target_time)
print('The profit corresponds to: ', float(df_trading.savings[mask]),' €')
print('The position was closed due to: ',df_trading.closing_mode[mask][i])

In [ ]:
df_predictions = rf_hourly_forecast(open_time,df,deadline,df_prices,s_opt)

In [ ]:
df_predictions = rf_load_hourly_forecast(open_time,df,deadline,df_prices,s_opt)

In [ ]:
if (int(df_trading.opening_type[mask] == 'buy')):
    best_idx = df_predictions.predictions.idxmax()
else:
    best_idx = df_predictions.predictions.idxmin()

In [ ]:
print(df_predictions.predictions[best_idx])
print(df_predictions.real_values[best_idx])

In [ ]:
np.max(df_predictions.predictions)

In [ ]:
profit_LG = []
forecasts_LG = []
for i in range(0,10):
    
    mask = df_trading.index == i
    
    selected_month = int(df_trading.open_position_time[mask].dt.month)
    selected_day = int(df_trading.open_position_time[mask].dt.day)
    selected_hour = int(df_trading.open_position_time[mask].dt.hour)
    selected_minute = int(df_trading.open_position_time[mask].dt.minute)

    selected_time = [selected_month, selected_day, selected_hour, selected_minute]

    plot_mask = (df_prices.month == selected_month) & ((df_prices.day == selected_day) | (df_prices.day == (selected_day + 1)))


    open_time = df_trading.open_position_time[mask][i]
    open_price = float(df_trading.price_at_open_time[mask])

    target_time = df_trading.target_time[mask][i]
    closing_price = float(df_trading.closing_price[mask])
    deadline = df_trading.datetime[mask][i]
    price_diff = abs(closing_price-open_price)
    
    df_predictions = rf_hourly_forecast(open_time,df_15s,deadline,df_prices,s_opt)
    forecasts_LG.append(df_predictions)
    
    df_predictions = df_predictions[df_predictions.timestamp < (deadline-pd.Timedelta(value=15, unit='min'))]
    
    if (int(df_trading.opening_type[mask] == 'buy')):
        best_idx = df_predictions.predictions.idxmax()
        profit = df_predictions.real_values[best_idx] - open_price
    else:
        best_idx = df_predictions.predictions.idxmin()
        profit = open_price - df_predictions.real_values[best_idx]
    
    profit_LG.append(profit)

## 3.2 Improved model: RF pre-trained and loaded at every iteration

In [ ]:
import joblib

### 3.2.1 Function that loads the random forests and estimates the predictions

In [ ]:
# the function takes into account 4 inputs:
# the position opening time
# the dataframe with the prices used to train the model
# the deadline to close the position
# the prices original dataset
# the hyperparameters of the RF

def rf_new_load_hourly_forecast(open_time,df,deadline,df_prices,s_opt):
    
    starting_month = open_time.month
    starting_day = open_time.day
    starting_hour = open_time.hour
    starting_minute = open_time.minute


    ending_month = deadline.month
    ending_day = deadline.day
    ending_hour = deadline.hour
    ending_minute = deadline.minute
    
    # The hours to forecast are all the hours between the opening time and the deadline.
    # The code differenciates if there are two days rather than 1
    
    if ending_day == starting_day:
        hours_forecast = range(starting_hour,(ending_hour+1))
    else:
        hours_forecast = np.concatenate([np.array(range(starting_hour,24)),np.array(range(0,ending_hour+1))])

    predictions = []
    real_values = []
    esse = []
    timeframe = []
    df_predictions = pd.DataFrame()
    
    # for each hour to be forecasted we will use the correspondent hourly model to perform the predictons
    for hour in hours_forecast:
        
        # the best hyperparameter for the selected hour
        s = s_opt.RMSE[hour]
        esse.append(s)

        selected_hour = hour
        
        if ((ending_day != starting_day) & (hour >= 12)):
            current_day = starting_day
        elif ((ending_day != starting_day) & (hour < 12)):
            current_day = ending_day

        else:
            current_day = starting_day
                
                
#         ### 1  CALCULATION OF TI
        sma =df.price.rolling(window=s).mean()
        mom = df.price.diff(s)
        rsi = pta.rsi(df.price, length = s)
        mstd =df.price.rolling(window=s).std()
        ema_1 = df.price.ewm(span=7, adjust=False).mean()
        ema_2 = df.price.ewm(span=14, adjust=False).mean()
        macd = ema_1 - ema_2
        BB_upper = sma + 2*mstd
        BB_lower = sma - 2*mstd
        ppcma = df.price.pct_change(periods=s)
        mad = (df.price-ppcma)/ppcma
        rolling_max = df.price.rolling(window=s).max()
        rolling_min = df.price.rolling(window=s).min()
        pr = ((rolling_max-df.price)/(rolling_max-rolling_min))*100
        A = rolling_max - rolling_min
        B = np.abs(rolling_max - df.price)
        C = np.abs(rolling_min - df.price)
        ABC=pd.DataFrame(A)
        ABC['B'] = B
        ABC['C'] = C
        MAX = ABC.max(axis=1)
        atr = MAX.rolling(window=s).mean()

        ### 2 CREATION OF A DATAFRAME OF TI

        features=pd.DataFrame(df.price)
        features['sma'] = sma
        features['mom'] = mom
        features['rsi'] = rsi
        features['mstd'] = mstd
        features['macd'] = macd
        features['BB_upper'] = BB_upper
        features['BB_lower'] = BB_lower
        features['ppcma'] = ppcma
        features['mad'] = mad
        features['pr'] = pr
        features['atr'] = atr
        features['month'] = df.month
        features['day'] = df.day
        features['weekday'] = df.timestamp.dt.weekday
        features['hour'] = df.hour
        features['minute'] = df.minute

        label = np.array(df.price[240:])
        label=np.concatenate([label,np.array(range(0,240))])
        features['label'] = label

        ### 3 CREATING HOURLY FEATURES!
        
        if selected_hour != 0:
            features_hour = features[features.hour == (selected_hour-1)]
            features_hour.replace([np.inf, -np.inf], np.nan, inplace=True)
            features_hour.fillna(-999, inplace = True)

            ### 4 CREATING A TRAINING AND A TESTING SET

            test_features = features_hour[(features_hour.month == 6)
                                      & (features_hour.day == current_day)]

            test_labels = features_hour.label[(features_hour.month == 6)
                                      & (features_hour.day == current_day)]
        else:
            
            features_hour = features[features.hour == 23]
            features_hour.replace([np.inf, -np.inf], np.nan, inplace=True)
            features_hour.fillna(-999, inplace = True)

            ### 4 CREATING A TRAINING AND A TESTING SET

            test_features = features_hour[(features_hour.month == 6)
                                      & (features_hour.day == (current_day-1))]

            test_labels = features_hour.label[(features_hour.month == 6)
                                      & (features_hour.day == (current_day-1))]
#         del train_features['label']
        del test_features['label']
        
        path = "C:/Users/luigi/OneDrive/2 University/4 ULB/0 THESIS/RF_models_CANTIERE_NAVALE/"+('rf' + str(hour))+".joblib"
        
        rf = joblib.load(path)

        ### 6 ESTIMATION OF THE PREDICTIONS AND OF THE ERRORS
        predictions = rf.predict(test_features)
        real_values = test_labels.values
        #predictions= np.concatenate([predictions, rf.predict(test_features)])
        #real_values = np.concatenate([real_values, test_labels.values])
        
        df_predictions_temp = pd.DataFrame(test_labels.index)
        df_predictions_temp['predictions'] = predictions
        df_predictions_temp['real_values'] = real_values
        
        df_predictions = pd.concat([df_predictions,df_predictions_temp])
        
#     plt.figure(figsize = (20,10))
#     plt.plot(df_predictions.timestamp,df_predictions.predictions,'r', label='predictions')
#     plt.plot(df_predictions.timestamp,df_predictions.real_values,'b', label='real values')
#     plt.legend(prop={'size': 20})
#     plt.title('Predictions of the RF models', size=30)
#     plt.ylabel('€', size=20)
#     plt.show()
    
    n = len(df_predictions.predictions)
    RMSE = np.sqrt(1/n*np.sum((df_predictions.predictions
                               -df_predictions.real_values)**2))
    print(RMSE)
    df_predictions.index = np.array(range(0,len(df_predictions.predictions)))
    return(df_predictions)
    

### 3.2.2 Normal strategy

In [ ]:
import timeit

In [ ]:
len(df_trading.index)

In [ ]:
start = timeit.default_timer()

# We create some lists of parameters we want to extract from the simulation
profit_LG = []
forecasts_LG = []
final_time_LG = []
final_prices_LG = []

# We iterate on all the positions in the testing dataset
for i in range(0,20):
    
    mask = df_trading.index == i
    
    selected_month = int(df_trading.open_position_time[mask].dt.month)
    selected_day = int(df_trading.open_position_time[mask].dt.day)
    selected_hour = int(df_trading.open_position_time[mask].dt.hour)
    selected_minute = int(df_trading.open_position_time[mask].dt.minute)

    selected_time = [selected_month, selected_day, selected_hour, selected_minute]

    plot_mask = (df_prices.month == selected_month) & ((df_prices.day == selected_day) | (df_prices.day == (selected_day + 1)))


    open_time = df_trading.open_position_time[mask][i]
    open_price = float(df_trading.price_at_open_time[mask])

    target_time = df_trading.target_time[mask][i]
    closing_price = float(df_trading.closing_price[mask])
    deadline = df_trading.datetime[mask][i]
    price_diff = abs(closing_price-open_price)
    
    # We use the previously created function to perform the forecasts and we save them in a list
    
    df_predictions = rf_new_load_hourly_forecast(open_time,df,deadline,df_prices,s_opt)
    forecasts_LG.append(df_predictions)
    # We select only the forecasts that happen before the deadline of each position
    
    df_predictions = df_predictions[(df_predictions.timestamp < (open_time+pd.Timedelta(value=60, unit='min'))) & (df_predictions.timestamp < deadline)]
    df_predictions = df_predictions[df_predictions.timestamp > open_time]
    
    
    # We estimate the profit. The closing price is estimated by selecting the best minute from the forecast
    # and then if there is more than one datapoint within that specific minute
    # a random value is extracted and considered
    
    if (int(df_trading.opening_type[mask] == 'buy')):
        best_idx = df_predictions.predictions.idxmax()
        
        best_month = df_predictions.timestamp[best_idx].month
        best_day = df_predictions.timestamp[best_idx].day
        best_hour = df_predictions.timestamp[best_idx].hour
        best_minute = df_predictions.timestamp[best_idx].minute

        best_mask = (df_prices.month == best_month) & (df_prices.day == best_day) & (df_prices.hour == best_hour) & (df_prices.minute == best_minute)
        random_order = ((df_prices[best_mask]))
        
        if len(random_order) == 0:
            final_price = closing_price
        else:
            
            random_order = random_order.sample(frac=1)
            final_price = random_order.price[0]
            final_time = random_order.index[0]
            profit = final_price - open_price
        
        
        #profit = df_predictions.real_values[best_idx] - open_price
    else:
        best_idx = df_predictions.predictions.idxmin()
        
        best_month = df_predictions.timestamp[best_idx].month
        best_day = df_predictions.timestamp[best_idx].day
        best_hour = df_predictions.timestamp[best_idx].hour
        best_minute = df_predictions.timestamp[best_idx].minute

        best_mask = (df_prices.month == best_month) & (df_prices.day == best_day) & (df_prices.hour == best_hour) & (df_prices.minute == best_minute)
        random_order = ((df_prices[best_mask]))
        
        if len(random_order) == 0:
            final_price = closing_price
        else:
            
            random_order = random_order.sample(frac=1)
            final_price = random_order.price[0]
            final_time = random_order.index[0]
            profit = open_price - final_price
        

    # We save the effective final price that has been considered and the profit made, which will be slightly different at every iteration   
        
    final_prices_LG.append(final_price)   
    final_time_LG.append(final_time)
    profit_LG.append(profit)
    
stop = timeit.default_timer()

print(np.sum(profit_LG))
print('Time: ', stop - start)


In [ ]:
start = timeit.default_timer()

# We create some lists of parameters we want to extract from the simulation
profit_LG = []
forecasts_LG = []
final_time_LG = []
final_prices_LG = []

# We iterate on all the positions in the testing dataset
for i in range(0,1):
    
    mask = df_trading.index == i
    
    selected_month = int(df_trading.open_position_time[mask].dt.month)
    selected_day = int(df_trading.open_position_time[mask].dt.day)
    selected_hour = int(df_trading.open_position_time[mask].dt.hour)
    selected_minute = int(df_trading.open_position_time[mask].dt.minute)

    selected_time = [selected_month, selected_day, selected_hour, selected_minute]

    plot_mask = (df_prices.month == selected_month) & ((df_prices.day == selected_day) | (df_prices.day == (selected_day + 1)))


    open_time = df_trading.open_position_time[mask][i]
    open_price = float(df_trading.price_at_open_time[mask])

    target_time = df_trading.target_time[mask][i]
    closing_price = float(df_trading.closing_price[mask])
    deadline = df_trading.datetime[mask][i]
    price_diff = abs(closing_price-open_price)
    
    # We use the previously created function to perform the forecasts and we save them in a list
    
    df_predictions = rf_new_load_hourly_forecast(open_time,df,deadline,df_prices,s_opt)
    forecasts_LG.append(df_predictions)
    # We select only the forecasts that happen before the deadline of each position
    
    df_predictions = df_predictions[(df_predictions.timestamp < (open_time+pd.Timedelta(value=60, unit='min'))) & (df_predictions.timestamp < deadline)]
    df_predictions = df_predictions[df_predictions.timestamp > open_time]
    
    
    # We estimate the profit. The closing price is estimated by selecting the best minute from the forecast
    # and then if there is more than one datapoint within that specific minute
    # a random value is extracted and considered
    
    if (int(df_trading.opening_type[mask] == 'buy')):
        best_idx = df_predictions.predictions.idxmax()
        
        best_month = df_predictions.timestamp[best_idx].month
        best_day = df_predictions.timestamp[best_idx].day
        best_hour = df_predictions.timestamp[best_idx].hour
        best_minute = df_predictions.timestamp[best_idx].minute

        best_mask = (df_prices.month == best_month) & (df_prices.day == best_day) & (df_prices.hour == best_hour) & (df_prices.minute == best_minute)
        random_order = ((df_prices[best_mask]))
        
        if len(random_order) == 0:
            final_price = closing_price
        else:
            
            random_order = random_order.sample(frac=1)
            final_price = random_order.price[0]
            final_time = random_order.index[0]
            profit = final_price - open_price
        
        
        #profit = df_predictions.real_values[best_idx] - open_price
    else:
        best_idx = df_predictions.predictions.idxmin()
        
        best_month = df_predictions.timestamp[best_idx].month
        best_day = df_predictions.timestamp[best_idx].day
        best_hour = df_predictions.timestamp[best_idx].hour
        best_minute = df_predictions.timestamp[best_idx].minute

        best_mask = (df_prices.month == best_month) & (df_prices.day == best_day) & (df_prices.hour == best_hour) & (df_prices.minute == best_minute)
        random_order = ((df_prices[best_mask]))
        
        if len(random_order) == 0:
            final_price = closing_price
        else:
            
            random_order = random_order.sample(frac=1)
            final_price = random_order.price[0]
            final_time = random_order.index[0]
            profit = open_price - final_price
        

    # We save the effective final price that has been considered and the profit made, which will be slightly different at every iteration   
        
    final_prices_LG.append(final_price)   
    final_time_LG.append(final_time)
    profit_LG.append(profit)
    
stop = timeit.default_timer()

print(np.sum(profit_LG))
print('Time: ', stop - start)


### 3. Extended strategy

In [ ]:
start = timeit.default_timer()

# We create some lists of parameters we want to extract from the simulation
profit_LG = []
forecasts_LG = []
final_time_LG = []
final_prices_LG = []
extended_strategy=[]
# We iterate on all the positions in the testing dataset
for i in range(0,595):
    
    mask = df_trading.index == i
    
    selected_month = int(df_trading.open_position_time[mask].dt.month)
    selected_day = int(df_trading.open_position_time[mask].dt.day)
    selected_hour = int(df_trading.open_position_time[mask].dt.hour)
    selected_minute = int(df_trading.open_position_time[mask].dt.minute)

    selected_time = [selected_month, selected_day, selected_hour, selected_minute]

    plot_mask = (df_prices.month == selected_month) & ((df_prices.day == selected_day) | (df_prices.day == (selected_day + 1)))


    open_time = df_trading.open_position_time[mask][i]
    open_price = float(df_trading.price_at_open_time[mask])

    target_time = df_trading.target_time[mask][i]
    closing_price = float(df_trading.closing_price[mask])
    deadline = df_trading.datetime[mask][i]
    price_diff = abs(closing_price-open_price)
    
    # We use the previously created function to perform the forecasts and we save them in a list
    
    df_predictions = rf_new_load_hourly_forecast(open_time,df,deadline,df_prices,s_opt)
    forecasts_LG.append(df_predictions)
    # We select only the forecasts that happen before the deadline of each position
    df_predictions_1 = df_predictions
    
    df_predictions = df_predictions[(df_predictions.timestamp < (open_time+pd.Timedelta(value=60, unit='min'))) & (df_predictions.timestamp < deadline)]
    df_predictions = df_predictions[df_predictions.timestamp > open_time]
    
    
    # We estimate the profit. The closing price is estimated by selecting the best minute from the forecast
    # and then if there is more than one datapoint within that specific minute
    # a random value is extracted and considered
    
    if (int(df_trading.opening_type[mask] == 'buy')):
        best_idx = df_predictions.predictions.idxmax()
        expected_price = df_predictions.predictions[best_idx]
        expected_profit = expected_price-open_price
        
        
        best_month = df_predictions.timestamp[best_idx].month
        best_day = df_predictions.timestamp[best_idx].day
        best_hour = df_predictions.timestamp[best_idx].hour
        best_minute = df_predictions.timestamp[best_idx].minute

        best_mask = (df_prices.month == best_month) & (df_prices.day == best_day) & (df_prices.hour == best_hour) & (df_prices.minute == best_minute)
        random_order = ((df_prices[best_mask]))
        
        if len(random_order) == 0:
            final_price = closing_price
        else:
            
            random_order = random_order.sample(frac=1)
            final_price = random_order.price[0]
            final_time = random_order.index[0]
            profit = final_price - open_price
        
        
        #profit = df_predictions.real_values[best_idx] - open_price
    else:
        best_idx = df_predictions.predictions.idxmin()
        expected_price = df_predictions.predictions[best_idx]
        expected_profit = open_price - expected_price
        
        best_month = df_predictions.timestamp[best_idx].month
        best_day = df_predictions.timestamp[best_idx].day
        best_hour = df_predictions.timestamp[best_idx].hour
        best_minute = df_predictions.timestamp[best_idx].minute

        best_mask = (df_prices.month == best_month) & (df_prices.day == best_day) & (df_prices.hour == best_hour) & (df_prices.minute == best_minute)
        random_order = ((df_prices[best_mask]))
        
        if len(random_order) == 0:
            final_price = closing_price
        else:
            
            random_order = random_order.sample(frac=1)
            final_price = random_order.price[0]
            final_time = random_order.index[0]
            profit = open_price - final_price
        
        
        
        
        
        
        
        
        
        
    if ((expected_profit < 0) & ((open_time + pd.Timedelta(value=120, unit='min'))<deadline)):
        
        df_predictions = df_predictions_1
        extended_strategy.append(1)
        df_predictions = df_predictions[(df_predictions.timestamp < (open_time + pd.Timedelta(value=120, unit='min'))) & (df_predictions.timestamp < deadline)]
        df_predictions = df_predictions[df_predictions.timestamp > (open_time + pd.Timedelta(value=60, unit='min'))]

        if (int(df_trading.opening_type[mask] == 'buy')):
            best_idx = df_predictions.predictions.idxmax()

            best_month = df_predictions.timestamp[best_idx].month
            best_day = df_predictions.timestamp[best_idx].day
            best_hour = df_predictions.timestamp[best_idx].hour
            best_minute = df_predictions.timestamp[best_idx].minute

            best_mask = (df_prices.month == best_month) & (df_prices.day == best_day) & (df_prices.hour == best_hour) & (df_prices.minute == best_minute)
            random_order = ((df_prices[best_mask]))

            if len(random_order) == 0:
                final_price = closing_price
            else:

                random_order = random_order.sample(frac=1)
                final_price = random_order.price[0]
                final_time = random_order.index[0]
                profit = final_price - open_price

        else:
            best_idx = df_predictions.predictions.idxmin()

            best_month = df_predictions.timestamp[best_idx].month
            best_day = df_predictions.timestamp[best_idx].day
            best_hour = df_predictions.timestamp[best_idx].hour
            best_minute = df_predictions.timestamp[best_idx].minute

            best_mask = (df_prices.month == best_month) & (df_prices.day == best_day) & (df_prices.hour == best_hour) & (df_prices.minute == best_minute)
            random_order = ((df_prices[best_mask]))

            if len(random_order) == 0:
                final_price = closing_price
            else:

                random_order = random_order.sample(frac=1)
                final_price = random_order.price[0]
                final_time = random_order.index[0]
                profit = open_price - final_price
    else:
        extended_strategy.append(0)
            
    # We save the effective final price that has been considered and the profit made, which will be slightly different at every iteration   
        
    final_prices_LG.append(final_price)   
    final_time_LG.append(final_time)
    profit_LG.append(profit)
    
stop = timeit.default_timer()

print(np.sum(profit_LG))
print('Time: ', stop - start)

In [ ]:
np.nansum(profit_LG)

In [ ]:
np.nansum(profit_LG)

### montecarlo simulation

In [ ]:
MC_profit = []
for m in range(0,15):

    # We create some lists of parameters we want to extract from the simulation
    profit_LG = []
    forecasts_LG = []
    final_time_LG = []
    final_prices_LG = []
    extended_strategy=[]
    # We iterate on all the positions in the testing dataset
    for i in range(0,595):

        mask = df_trading.index == i

        selected_month = int(df_trading.open_position_time[mask].dt.month)
        selected_day = int(df_trading.open_position_time[mask].dt.day)
        selected_hour = int(df_trading.open_position_time[mask].dt.hour)
        selected_minute = int(df_trading.open_position_time[mask].dt.minute)

        selected_time = [selected_month, selected_day, selected_hour, selected_minute]

        plot_mask = (df_prices.month == selected_month) & ((df_prices.day == selected_day) | (df_prices.day == (selected_day + 1)))


        open_time = df_trading.open_position_time[mask][i]
        open_price = float(df_trading.price_at_open_time[mask])

        target_time = df_trading.target_time[mask][i]
        closing_price = float(df_trading.closing_price[mask])
        deadline = df_trading.datetime[mask][i]
        price_diff = abs(closing_price-open_price)

        # We use the previously created function to perform the forecasts and we save them in a list

        df_predictions = rf_new_load_hourly_forecast(open_time,df,deadline,df_prices,s_opt)
        forecasts_LG.append(df_predictions)
        # We select only the forecasts that happen before the deadline of each position

        df_predictions = df_predictions[(df_predictions.timestamp < (open_time+pd.Timedelta(value=60, unit='min'))) & (df_predictions.timestamp < deadline)]
        df_predictions = df_predictions[df_predictions.timestamp > open_time]


        # We estimate the profit. The closing price is estimated by selecting the best minute from the forecast
        # and then if there is more than one datapoint within that specific minute
        # a random value is extracted and considered

        if (int(df_trading.opening_type[mask] == 'buy')):
            best_idx = df_predictions.predictions.idxmax()

            best_month = df_predictions.timestamp[best_idx].month
            best_day = df_predictions.timestamp[best_idx].day
            best_hour = df_predictions.timestamp[best_idx].hour
            best_minute = df_predictions.timestamp[best_idx].minute

            best_mask = (df_prices.month == best_month) & (df_prices.day == best_day) & (df_prices.hour == best_hour) & (df_prices.minute == best_minute)
            random_order = ((df_prices[best_mask]))

            if len(random_order) == 0:
                final_price = closing_price
            else:

                random_order = random_order.sample(frac=1)
                final_price = random_order.price[0]
                final_time = random_order.index[0]
                profit = final_price - open_price


            #profit = df_predictions.real_values[best_idx] - open_price
        else:
            best_idx = df_predictions.predictions.idxmin()

            best_month = df_predictions.timestamp[best_idx].month
            best_day = df_predictions.timestamp[best_idx].day
            best_hour = df_predictions.timestamp[best_idx].hour
            best_minute = df_predictions.timestamp[best_idx].minute

            best_mask = (df_prices.month == best_month) & (df_prices.day == best_day) & (df_prices.hour == best_hour) & (df_prices.minute == best_minute)
            random_order = ((df_prices[best_mask]))

            if len(random_order) == 0:
                final_price = closing_price
            else:

                random_order = random_order.sample(frac=1)
                final_price = random_order.price[0]
                final_time = random_order.index[0]
                profit = open_price - final_price


        # We save the effective final price that has been considered and the profit made, which will be slightly different at every iteration   

        final_prices_LG.append(final_price)   
        final_time_LG.append(final_time)
        profit_LG.append(profit)

    stop = timeit.default_timer()

    print(np.sum(profit_LG))

    MC_profit.append(np.nansum(profit_LG))

In [ ]:
MC_profit=pd.DataFrame(MC_profit)
MC_profit.to_csv(r'C:\Users\luigi\OneDrive\2 University\4 ULB\0 THESIS\FINAL_SIMULATIONS\MC1.csv')

In [ ]:
MC_profit

In [ ]:
extended_strategy

In [ ]:
np.sum(df_trading.savings[0:595])

In [ ]:
np.mean(MC_profit)

### 3.2.3 Display of one position of the testing period extracted from the simulation

In [ ]:
i = 43
mask = df_trading.index == i


# We define some plotting parameters
selected_month = int(df_trading.open_position_time[mask].dt.month)
selected_day = int(df_trading.open_position_time[mask].dt.day)
selected_hour = int(df_trading.open_position_time[mask].dt.hour)
selected_minute = int(df_trading.open_position_time[mask].dt.minute)

selected_time = [selected_month, selected_day, selected_hour, selected_minute]

plot_mask = (df_prices.month == selected_month) & ((df_prices.day == selected_day) | (df_prices.day == (selected_day + 1)))


open_time = df_trading.open_position_time[mask][i]
open_price = float(df_trading.price_at_open_time[mask])

target_time = df_trading.target_time[mask][i]
closing_price = float(df_trading.closing_price[mask])
deadline = df_trading.datetime[mask][i]

# We want to have the green line above the red when we make a profit.
# If we have a long position (buy then sell) we will have the green line for the sell price and the red line for the buy price.
# If we have a short position (sell then buy) we will have the green line for the buy price and the red line for the sell price.
      
if (int(df_trading.opening_type[mask] == 'buy')):
    plt.figure(figsize = (20,10))
    
    plt.plot(df_prices.price[plot_mask],alpha=0.3, c ='b' ,label = 'prices')
    plt.plot(forecasts_LG[i].timestamp,forecasts_LG[i].predictions,c='navy',alpha=0.5, label = 'prices forecast')
    
    plt.axvline(x=(open_time), c='r', label='opening time',linewidth=2.0)
    plt.axhline(y=open_price, color='r', linestyle='--', label='opening price (buy)')

    plt.axvline(x=deadline, c='black', label='deadline',linewidth=5.0)
    
    plt.axvline(x=target_time, c='g', label='closing time',linewidth=2.0)
    plt.axhline(y=closing_price, color='g', linestyle='--', label='closing price (sell)')
    
    plt.axvline(x=final_time_LG[i], c='indigo', label='new closing time',linewidth=2.0)
    plt.axhline(y=final_prices_LG[i], color='indigo', linestyle='--', label='new closing price (sell)')
                      
                      
    plt.ylim(min(open_price,closing_price,final_prices_LG[i])-10,max(open_price,closing_price,final_prices_LG[i])+10)
    plt.xlim(forecasts_LG[i].timestamp[0],forecasts_LG[i].timestamp[len(forecasts_LG[i].timestamp)-1])
    
    if (open_price-closing_price < 0):
        
        plt.fill_between(df_prices.index[plot_mask],
                         closing_price,
                        open_price, 
                        color='g', alpha=.15)
    else:
        plt.fill_between(df_prices.index[plot_mask],
                         closing_price,
                        open_price, 
                        color='r', alpha=.15)
    plt.title('Long trading position', size=30)
    plt.legend(prop={'size': 20}, loc = 'upper left')
    plt.ylabel('€/MWh', size=20)
    plt.show()

else:
    plt.figure(figsize = (20,10))                    
    plt.plot(df_prices.price[plot_mask],alpha=0.2, c ='b' ,label = 'prices')
    plt.plot(forecasts_LG[i].timestamp,forecasts_LG[i].predictions,c='navy',alpha=0.6, label = 'prices forecast')
    
    plt.axvline(x=(open_time), c='g', label = 'opening time',linewidth=2.0)
    plt.axhline(y=open_price, color='g', linestyle='--', label='opening price (sell)')

    plt.axvline(x=deadline, c='black', label = 'deadline',linewidth=5.0)
    
    plt.axvline(x=target_time, c='r', label='closing time',linewidth=2.0)
    plt.axhline(y=closing_price, color='r', linestyle='--', label='closing price (buy)')
    
    plt.axvline(x=final_time_LG[i], c='indigo', label='new closing time',linewidth=2.0)
    plt.axhline(y=final_prices_LG[i], color='indigo', linestyle='--', label='new closing price (buy)')
    
    plt.axvline(x=deadline, c='y')
                      
    
    plt.ylim(min(open_price,closing_price,final_prices_LG[i])-10,max(open_price,closing_price,final_prices_LG[i])+10)
    plt.xlim(forecasts_LG[i].timestamp[0],forecasts_LG[i].timestamp[len(forecasts_LG[i].timestamp)-1])
    if (open_price-closing_price < 0):
        
        plt.fill_between(df_prices.index[plot_mask],
                         closing_price,
                        open_price, 
                        color='r', alpha=.15)
    else:
        plt.fill_between(df_prices.index[plot_mask],
                         closing_price,
                        open_price, 
                        color='g', alpha=.15)
                      
    plt.legend(prop={'size': 20}, loc = 'upper left')
    plt.title('Short trading position', size=30)
    plt.ylabel('€/MWh', size=20)
    plt.show()
    
print('The operation is: ',df_trading.opening_type[mask][i])
print('Opening price: ',open_price,' at ',open_time,' , closing price: ', closing_price,' at ', target_time)
print('Deadline: ',deadline,', New closing time: ', final_time_LG[i] )
print('The profit corresponds to: ', float(df_trading.savings[i]),' €')
print('The new profit corresponds to: ', round(float(profit_LG[i]),2),' €')
print('The position was closed due to: ',df_trading.closing_mode[mask][i])

In [ ]:
i = 12
mask = df_trading.index == i


# We define some plotting parameters
selected_month = int(df_trading.open_position_time[mask].dt.month)
selected_day = int(df_trading.open_position_time[mask].dt.day)
selected_hour = int(df_trading.open_position_time[mask].dt.hour)
selected_minute = int(df_trading.open_position_time[mask].dt.minute)

selected_time = [selected_month, selected_day, selected_hour, selected_minute]

plot_mask = (df_prices.month == selected_month) & ((df_prices.day == selected_day) | (df_prices.day == (selected_day + 1)))


open_time = df_trading.open_position_time[mask][i]
open_price = float(df_trading.price_at_open_time[mask])

target_time = df_trading.target_time[mask][i]
closing_price = float(df_trading.closing_price[mask])
deadline = df_trading.datetime[mask][i]

# We want to have the green line above the red when we make a profit.
# If we have a long position (buy then sell) we will have the green line for the sell price and the red line for the buy price.
# If we have a short position (sell then buy) we will have the green line for the buy price and the red line for the sell price.
      
if (int(df_trading.opening_type[mask] == 'buy')):
    plt.figure(figsize = (20,10))
    
    plt.plot(df_prices.price[plot_mask],alpha=0.3, c ='b' ,label = 'prices')
    plt.plot(forecasts_LG[i].timestamp,forecasts_LG[i].predictions,c='navy',alpha=0.5, label = 'prices forecast')
    
    plt.axvline(x=(open_time), c='r', label='opening time',linewidth=2.0)
    plt.axhline(y=open_price, color='r', linestyle='--', label='opening price (buy)')

    plt.axvline(x=deadline, c='black', label='deadline',linewidth=5.0)
    
    plt.axvline(x=target_time, c='g', label='closing time',linewidth=2.0)
    plt.axhline(y=closing_price, color='g', linestyle='--', label='closing price (sell)')
    
    plt.axvline(x=final_time_LG[i], c='indigo', label='new closing time',linewidth=2.0)
    plt.axhline(y=final_prices_LG[i], color='indigo', linestyle='--', label='new closing price (sell)')
                      
                      
    plt.ylim(min(open_price,closing_price,final_prices_LG[i])-10,max(open_price,closing_price,final_prices_LG[i])+10)
    plt.xlim(forecasts_LG[i].timestamp[0],forecasts_LG[i].timestamp[len(forecasts_LG[i].timestamp)-1])
    
    if (open_price-closing_price < 0):
        
        plt.fill_between(df_prices.index[plot_mask],
                         closing_price,
                        open_price, 
                        color='g', alpha=.15)
    else:
        plt.fill_between(df_prices.index[plot_mask],
                         closing_price,
                        open_price, 
                        color='r', alpha=.15)
    plt.title('Long trading position', size=30)
    plt.legend(prop={'size': 20}, loc = 'upper left')
    plt.ylabel('€/MWh', size=20)
    plt.show()

else:
    plt.figure(figsize = (20,10))                    
    plt.plot(df_prices.price[plot_mask],alpha=0.2, c ='b' ,label = 'prices')
    plt.plot(forecasts_LG[i].timestamp,forecasts_LG[i].predictions,c='navy',alpha=0.6, label = 'prices forecast')
    
    plt.axvline(x=(open_time), c='g', label = 'opening time',linewidth=2.0)
    plt.axhline(y=open_price, color='g', linestyle='--', label='opening price (sell)')

    plt.axvline(x=deadline, c='black', label = 'deadline',linewidth=5.0)
    
    plt.axvline(x=target_time, c='r', label='closing time',linewidth=2.0)
    plt.axhline(y=closing_price, color='r', linestyle='--', label='closing price (buy)')
    
    plt.axvline(x=final_time_LG[i], c='indigo', label='new closing time',linewidth=2.0)
    plt.axhline(y=final_prices_LG[i], color='indigo', linestyle='--', label='new closing price (buy)')
    
    plt.axvline(x=deadline, c='y')
                      
    
    plt.ylim(min(open_price,closing_price,final_prices_LG[i])-10,max(open_price,closing_price,final_prices_LG[i])+10)
    plt.xlim(forecasts_LG[i].timestamp[0],forecasts_LG[i].timestamp[len(forecasts_LG[i].timestamp)-1])
    if (open_price-closing_price < 0):
        
        plt.fill_between(df_prices.index[plot_mask],
                         closing_price,
                        open_price, 
                        color='r', alpha=.15)
    else:
        plt.fill_between(df_prices.index[plot_mask],
                         closing_price,
                        open_price, 
                        color='g', alpha=.15)
                      
    plt.legend(prop={'size': 20}, loc = 'upper left')
    plt.title('Short trading position', size=30)
    plt.ylabel('€/MWh', size=20)
    plt.show()
    
print('The operation is: ',df_trading.opening_type[mask][i])
print('Opening price: ',open_price,' at ',open_time,' , closing price: ', closing_price,' at ', target_time)
print('Deadline: ',deadline,', New closing time: ', final_time_LG[i] )
print('The profit corresponds to: ', float(df_trading.savings[i]),' €')
print('The new profit corresponds to: ', round(float(profit_LG[i]),2),' €')
print('The position was closed due to: ',df_trading.closing_mode[mask][i])

In [ ]:
np.sum(profit_LG>df_trading.savings[0:595])

In [ ]:
extended_strategy

### 3.2.4 Histogram of the profits of the old and new strategies compared

In [ ]:
N = len(profit_LG)
h = sct.iqr(profit_LG, nan_policy='omit') * 2 * N**(-1/3)
n_bins = int(np.floor((np.nanmax(profit_LG)-np.nanmin(profit_LG))/h))

In [ ]:
N = len(profit_LG)
N

In [ ]:
#i = 100
N = len(profit_LG)
original_savings = df_trading.savings[0:N]
h = sct.iqr(profit_LG, nan_policy='omit') * 2 * N**(-1/3)
n_bins = int(np.floor((np.nanmax(profit_LG)-np.nanmin(profit_LG))/h))

plt.figure(figsize = (20,10)) 
plt.hist(profit_LG, bins = n_bins,alpha=0.5, label='profit RF forecast', color = 'b')


# N = len(df_trading.savings)
# h = sct.iqr(df_trading.savings, nan_policy='omit') * 2 * N**(-1/3)
# n_bins = int(np.floor((np.nanmax(df_trading.savings)-np.nanmin(df_trading.savings))/h)) 
plt.hist(original_savings, bins = n_bins,alpha=0.5, label='profit original trading',color = 'darkorange')
#plt.hist(profit_LG, bins = n_bins, alpha=0.8, label='profit Luigi')

plt.axvline(x=np.mean(original_savings),c='r', label='mean profit original trading')
plt.axvline(x=np.nanmean(profit_LG),c='b',label='mean RF forecast')

plt.title('Comparison of the histograms of the profits (original vs RF-based model)', size=30)
plt.xlabel('€/MWh', size=20)
#plt.xlim(-50,50)
plt.ylabel('frequency', size=20)
plt.legend(prop={'size': 20})

plt.show()
basic_stats = sct.describe(original_savings,nan_policy='omit')
print('The total profit of the original strategy is: ',round(np.nansum(original_savings),2),' €, with a mean profit of: ',round(np.nanmean(original_savings),2),' €')
print('The total profit of the new strategy is: ',round(np.nansum(profit_LG),2),' €, with a mean profit of: ',round(np.nanmean(profit_LG),2),' €')
print('Improvement of: ',round(round(np.nansum(profit_LG),2)/round(np.nansum(original_savings),2),2)*100,' %')

In [ ]:
basic_stats = sct.describe(original_savings,nan_policy='omit')
print(basic_stats)

In [ ]:
basic_stats = sct.describe(profit_LG,nan_policy='omit')
print(basic_stats)

# 4 Trading strategy with stochastic volatility

In [ ]:
import sdepy
from sklearn.linear_model import LinearRegression
number_processes = 10

## 4.1 Test with one trading position, to tune the models

In [ ]:
i = 0
mask = df_trading.index == i

open_time = df_trading.open_position_time[mask][i]
open_price = float(df_trading.price_at_open_time[mask])

target_time = df_trading.target_time[mask][i]
closing_price = float(df_trading.closing_price[mask])
deadline = df_trading.datetime[mask][i]

### 4.1.1 Linear regression to reverse engineer the parameters

In [ ]:
mask_train = (df.timestamp > (open_time - pd.Timedelta(value=12960, unit='min'))) & (df.timestamp < open_time)

df_train = df[mask_train]

model = LinearRegression(fit_intercept = True, normalize = False)
x2 = np.array([[0]])
x1 = np.array([df_train.price]).T
x1 = x1[0:(len(x1)-1)]
x1 = np.concatenate((x2, x1))
y = np.array([df_train.price]).T

x1 = x1[1:len(x1)]
y = y[1:len(y)]

plt.figure(figsize = (20,10))
plt.scatter(x1,y)
plt.ylabel('price at time ti [€/MWh]', size=20)
plt.xlabel('price at time ti-1 [€/MWh]', size=20)
plt.title('Scatter plot of prices linear correlation', size=30)

In [ ]:
model.fit(x1, y)
r_sq = model.score(x1, y)
a = model.coef_[0]
b = model.intercept_
prediction = model.predict(x1)
residual = (y - prediction)
sd_eta = np.std(residual)

delta = 1
lambda_calc = -np.log(a)/delta
#print('estimated lambda:', lambda_calc)

mu_calc = b/(1-a)
#print('estimated mu: ', mu_calc)

sigma_calc = sd_eta*np.sqrt((-2*np.log(a))/(delta*(1-a**2)))
print(r_sq)

In [ ]:
plt.figure(figsize = (20,10))
plt.scatter(x1,y, label = 'original dataset')
plt.scatter(x1,prediction, label = 'prediction')
plt.ylabel('price at time ti [€/MWh]', size=20)
plt.xlabel('price at time ti-1 [€/MWh]', size=20)
plt.title('Predictions of the linear regression', size=30)
plt.legend(prop={'size': 30})
plt.show()

### 4.1.2 Display of the influence of a selected number of processes

In [ ]:
process = sdepy.ornstein_uhlenbeck_process(paths=number_processes, 
                                 vshape=(), 
                                 dtype=None, 
                                 steps=None, 
                                 i0=0, 
                                 info=None, 
                                 getinfo=True, 
                                 method='euler', 
                                 x0=open_price, 
                                 theta=mu_calc, 
                                 k=lambda_calc, 
                                 sigma=sigma_calc, 
                                 dw=None, 
                                 corr=None, 
                                 rho=None)


length_x = np.sum((df.timestamp > open_time) & (df.timestamp < deadline))
time = df.timestamp[(df.timestamp > open_time) & (df.timestamp < deadline)]
x_ou = np.array(range(0,length_x))

OU = []

plt.figure(figsize = (20,10))
for i in range(0,number_processes):
    ou = process(x_ou)[:,i]
    OU.append(ou)
    plt.plot(time, ou,alpha=0.7)
    
plt.plot(df.price[time],'b', label='real series')
plt.ylabel('€/MWh', size=20)
plt.title('Ornstein-Uhlenbeck processes generation (10 processes)', size=30)
plt.legend(prop={'size': 30})
plt.show()

In [ ]:
OU = pd.DataFrame(OU)
mean_OU = OU.mean(axis=0)

In [ ]:
plt.figure(figsize = (20,10))
plt.plot(time,mean_OU,'r', label = 'Mean of the OU processes')
plt.plot(df.price[time],'b', label = 'Real series of prices')
plt.ylabel('€/MWh', size=20)
plt.title('Ornstein-Uhlenbeck processes generation (10 processes)', size=30)
plt.legend(prop={'size': 30})
plt.show()

In [ ]:
n = len(mean_OU)
RMSE_i = np.sqrt(1/n*np.sum((mean_OU-df.price[time].values)**2))
mape = np.nanmean(np.abs(mean_OU - df.price[time].values)/np.abs(df.price[time].values))  # MAPE
mae = np.nanmean(np.abs(mean_OU - df.price[time].values))
                             
print('RMSE: ',round(RMSE_i,2))
print('MAPE: ',round(mape,2))
print('MAE: ',round(mae,2))

### 4.1.3 Tuning of the training subset (number of days of training)

In [ ]:
RMSE =[]
for i in range(1,30):
    
    mask_train = (df.timestamp > (open_time - pd.Timedelta(value=1440*i, unit='min'))) & (df.timestamp < open_time)

    df_train = df[mask_train]

    model = LinearRegression(fit_intercept = True, normalize = False)
    x2 = np.array([[0]])
    x1 = np.array([df_train.price]).T
    x1 = x1[0:(len(x1)-1)]
    x1 = np.concatenate((x2, x1))
    y = np.array([df_train.price]).T

    x1 = x1[1:len(x1)]
    y = y[1:len(y)]
    model.fit(x1, y)
    a = model.coef_[0]
    b = model.intercept_
    prediction = model.predict(x1)
    residual = (y - prediction)
    sd_eta = np.std(residual)

    delta = 1
    lambda_calc = -np.log(a)/delta
    #print('estimated lambda:', lambda_calc)

    mu_calc = b/(1-a)
    #print('estimated mu: ', mu_calc)

    sigma_calc = sd_eta*np.sqrt((-2*np.log(a))/(delta*(1-a**2)))
    process = sdepy.ornstein_uhlenbeck_process(paths=number_processes, 
                                 vshape=(), 
                                 dtype=None, 
                                 steps=None, 
                                 i0=0, 
                                 info=None, 
                                 getinfo=True, 
                                 method='euler', 
                                 x0=open_price, 
                                 theta=mu_calc, 
                                 k=lambda_calc, 
                                 sigma=sigma_calc, 
                                 dw=None, 
                                 corr=None, 
                                 rho=None)


    length_x = np.sum((df.timestamp > open_time) & (df.timestamp < deadline))
    time = df.timestamp[(df.timestamp > open_time) & (df.timestamp < deadline)]
    x_ou = np.array(range(0,length_x))

    OU = []
    for i in range(0,number_processes):
        ou = process(x_ou)[:,i]
        OU.append(ou)
    #     plt.plot(time, ou)

    #plt.plot(df.price[time])
    OU = pd.DataFrame(OU)
    mean_OU = OU.mean(axis=0)
    n = len(mean_OU)
    RMSE_i = np.sqrt(1/n*np.sum((mean_OU
                                   -df.price[time].values)**2))
    RMSE.append(RMSE_i)

In [ ]:
plt.figure(figsize = (20,10))
plt.plot(RMSE, label='RMSE')
plt.ylabel('RMSE', size=20)
plt.xlabel('days of data for training', size=20)
plt.title('Tuning of the training period for OU process and RMSE', size=30)
plt.legend(prop={'size': 30})

### 4.1.4 Tuning of the numberof processes

In [ ]:
plt.figure(figsize = (20,10))
RMSE=[]
MAPE=[]
MAE=[]
for number_processes in range(1,50):
    process = sdepy.ornstein_uhlenbeck_process(paths=number_processes, 
                                     vshape=(), 
                                     dtype=None, 
                                     steps=None, 
                                     i0=0, 
                                     info=None, 
                                     getinfo=True, 
                                     method='euler', 
                                     x0=open_price, 
                                     theta=mu_calc, 
                                     k=lambda_calc, 
                                     sigma=sigma_calc, 
                                     dw=None, 
                                     corr=None, 
                                     rho=None)


    length_x = np.sum((df.timestamp > open_time) & (df.timestamp < deadline))
    time = df.timestamp[(df.timestamp > open_time) & (df.timestamp < deadline)]
    x_ou = np.array(range(0,length_x))

    OU = []

    
    for i in range(0,number_processes):
        ou = process(x_ou)[:,i]
        OU.append(ou)
        
    OU = pd.DataFrame(OU)
    mean_OU = OU.mean(axis=0)
    plt.plot(time,mean_OU, alpha=0.7)
    n = len(mean_OU)
    RMSE_i = np.sqrt(1/n*np.sum((mean_OU-df.price[time].values)**2))
    mape = np.nanmean(np.abs(mean_OU - df.price[time].values)/np.abs(df.price[time].values))  # MAPE
    mae = np.nanmean(np.abs(mean_OU - df.price[time].values))
    
    RMSE.append(RMSE_i)
    MAPE.append(mape)
    MAE.append(mae)
    
plt.plot(df.price[time],'b', label = 'Real series of prices',linewidth=5.0)    
plt.ylabel('€/MWh', size=20)
plt.title('Mean Ornstein-Uhlenbeck processes (from 1 to 50 processes)', size=30) 
plt.legend(prop={'size': 30})
plt.show()

In [ ]:
plt.figure(figsize = (10,5))
plt.plot(MAPE)
#plt.ylabel('€/MWh', size=20)
plt.xlabel('n. processes', size=15)
plt.title('MAPE: trend with number of processes', size=15) 
#plt.legend(prop={'size': 30})
plt.show()

In [ ]:

plt.figure(figsize = (10,5))
plt.plot(MAE)
plt.ylabel('€/MWh', size=20)
plt.xlabel('n. processes', size=15)
plt.title('MAE: trend with number of processes', size=15) 
#plt.legend(prop={'size': 30})
plt.show()

In [ ]:
plt.figure(figsize = (10,5))
plt.plot(RMSE)
plt.ylabel('€/MWh', size=20)
plt.xlabel('n. processes', size=15)
plt.title('RMSE: trend with number of processes', size=15) 
#plt.legend(prop={'size': 30})
plt.show()

## 4.3 Trading with OU

In [ ]:
def OU_hourly_forecast(open_time,df,deadline,df_prices):
    
    number_processes = 30
    
    starting_month = open_time.month
    starting_day = open_time.day
    starting_hour = open_time.hour
    starting_minute = open_time.minute


    ending_month = deadline.month
    ending_day = deadline.day
    ending_hour = deadline.hour
    ending_minute = deadline.minute
    
    # The hours to forecast are all the hours between the opening time and the deadline.
    # The code differenciates if there are two days rather than 1
    
    if ending_day == starting_day:
        hours_forecast = range(starting_hour,(ending_hour+1))
    else:
        hours_forecast = np.concatenate([np.array(range(starting_hour,24)),np.array(range(0,ending_hour+1))])

    predictions = []
    real_values = []
    esse = []
    timeframe = []
    df_predictions = pd.DataFrame()
    
    
    mask_train = (df.timestamp > (open_time - pd.Timedelta(value=12960, unit='min'))) & (df.timestamp < open_time)
    df_train = df[mask_train]
    model = LinearRegression(fit_intercept = True, normalize = False)
    x2 = np.array([[0]])
    x1 = np.array([df_train.price]).T
    x1 = x1[0:(len(x1)-1)]
    x1 = np.concatenate((x2, x1))
    y = np.array([df_train.price]).T
    x1 = x1[1:len(x1)]
    y = y[1:len(y)]
    
    model.fit(x1, y)
    r_sq = model.score(x1, y)
    a = model.coef_[0]
    b = model.intercept_
    prediction = model.predict(x1)
    residual = (y - prediction)
    sd_eta = np.std(residual)

    delta = 1
    lambda_calc = -np.log(a)/delta
    #print('estimated lambda:', lambda_calc)

    mu_calc = b/(1-a)
    #print('estimated mu: ', mu_calc)

    sigma_calc = sd_eta*np.sqrt((-2*np.log(a))/(delta*(1-a**2)))

    process = sdepy.ornstein_uhlenbeck_process(paths=number_processes, 
                                 vshape=(), 
                                 dtype=None, 
                                 steps=None, 
                                 i0=0, 
                                 info=None, 
                                 getinfo=True, 
                                 method='euler', 
                                 x0=open_price, 
                                 theta=mu_calc, 
                                 k=lambda_calc, 
                                 sigma=sigma_calc, 
                                 dw=None, 
                                 corr=None, 
                                 rho=None)


    length_x = np.sum((df.timestamp > open_time) & (df.timestamp < deadline))
    time = df.timestamp[(df.timestamp > open_time) & (df.timestamp < deadline)]
    x_ou = np.array(range(0,length_x))

    OU = []
    for i in range(0,number_processes):
        ou = process(x_ou)[:,i]
        OU.append(ou)
    #     plt.plot(time, ou)

    #plt.plot(df.price[time])
    OU = pd.DataFrame(OU)
    mean_OU = OU.mean(axis=0)
    
    n = len(mean_OU)
    RMSE_i = np.sqrt(1/n*np.sum((mean_OU
                                   -df.price[time].values)**2))
    RMSE.append(RMSE_i)

    return(mean_OU)

In [ ]:
final_prices_LG=[]   
final_time_LG=[]
profit_LG=[]
RMSE=[]

for i in range(0,595):
    mask = df_trading.index == i

    selected_month = int(df_trading.open_position_time[mask].dt.month)
    selected_day = int(df_trading.open_position_time[mask].dt.day)
    selected_hour = int(df_trading.open_position_time[mask].dt.hour)
    selected_minute = int(df_trading.open_position_time[mask].dt.minute)

    selected_time = [selected_month, selected_day, selected_hour, selected_minute]

    plot_mask = (df_prices.month == selected_month) & ((df_prices.day == selected_day) | (df_prices.day == (selected_day + 1)))


    open_time = df_trading.open_position_time[mask][i]
    open_price = float(df_trading.price_at_open_time[mask])

    target_time = df_trading.target_time[mask][i]
    closing_price = float(df_trading.closing_price[mask])
    deadline = df_trading.datetime[mask][i]

    time = df.timestamp[(df.timestamp > open_time) & (df.timestamp < deadline)]
    
    df_predictions = pd.DataFrame()
    df_predictions['predictions'] = OU_hourly_forecast(open_time,df,deadline,df_prices)
    df_predictions['timestamp'] = time.values
    
#     RMSE_i = np.sqrt(1/n*np.sum((prediction-df.price[time].values)**2))
#     RMSE.append(RMSE_i)

    forecasts_LG.append(df_predictions)
    # We select only the forecasts that happen before the deadline of each position
    
    df_predictions = df_predictions[df_predictions.timestamp < (deadline-pd.Timedelta(value=30, unit='min'))]
    df_predictions = df_predictions[df_predictions.timestamp > open_time]
    
    
    # We estimate the profit. The closing price is estimated by selecting the best minute from the forecast
    # and then if there is more than one datapoint within that specific minute
    # a random value is extracted and considered
    
    if (int(df_trading.opening_type[mask] == 'buy')):
        best_idx = df_predictions.predictions.idxmax()
        if pd.isna(best_idx) == True:
            final_price = closing_price
        else:
            
            best_month = df_predictions.timestamp[best_idx].month
            best_day = df_predictions.timestamp[best_idx].day
            best_hour = df_predictions.timestamp[best_idx].hour
            best_minute = df_predictions.timestamp[best_idx].minute

            best_mask = (df_prices.month == best_month) & (df_prices.day == best_day) & (df_prices.hour == best_hour) & (df_prices.minute == best_minute)
            random_order = ((df_prices[best_mask]))

            if len(random_order) == 0:
                final_price = closing_price
            else:

                random_order = random_order.sample(frac=1)
                final_price = random_order.price[0]
                final_time = random_order.index[0]
                profit = final_price - open_price
        
        
        #profit = df_predictions.real_values[best_idx] - open_price
    else:
        best_idx = df_predictions.predictions.idxmin()
        if pd.isna(best_idx) == True:
            final_price = closing_price
        else:
            
            best_month = df_predictions.timestamp[best_idx].month
            best_day = df_predictions.timestamp[best_idx].day
            best_hour = df_predictions.timestamp[best_idx].hour
            best_minute = df_predictions.timestamp[best_idx].minute

            best_mask = (df_prices.month == best_month) & (df_prices.day == best_day) & (df_prices.hour == best_hour) & (df_prices.minute == best_minute)
            random_order = ((df_prices[best_mask]))

            if len(random_order) == 0:
                final_price = closing_price
            else:

                random_order = random_order.sample(frac=1)
                final_price = random_order.price[0]
                final_time = random_order.index[0]
                profit = open_price - final_price
        

    # We save the effective final price that has been considered and the profit made, which will be slightly different at every iteration   
        
    final_prices_LG.append(final_price)   
    final_time_LG.append(final_time)
    profit_LG.append(profit)

In [ ]:
np.sum(profit_LG)
np.mean(profit_LG)

In [ ]:
len(profit_LG)

In [ ]:
#i = 100
N = len(profit_LG)
original_savings = df_trading.savings[0:N]
h = sct.iqr(profit_LG, nan_policy='omit') * 2 * N**(-1/3)
n_bins = int(np.floor((np.nanmax(profit_LG)-np.nanmin(profit_LG))/h))

plt.figure(figsize = (20,10)) 
plt.hist(profit_LG, bins = n_bins,alpha=0.5, label='profit OU forecast', color = 'DarkViolet')


# N = len(df_trading.savings)
# h = sct.iqr(df_trading.savings, nan_policy='omit') * 2 * N**(-1/3)
# n_bins = int(np.floor((np.nanmax(df_trading.savings)-np.nanmin(df_trading.savings))/h)) 
plt.hist(original_savings, bins = n_bins,alpha=0.5, label='profit original trading',color = 'darkorange')
#plt.hist(profit_LG, bins = n_bins, alpha=0.8, label='profit Luigi')

plt.axvline(x=np.mean(original_savings),c='r', label='mean profit original trading')
plt.axvline(x=np.nanmean(profit_LG),c='DarkViolet',label='mean OU forecast')

plt.title('Comparison of the histograms of the profits (original vs OU-based model)', size=30)
plt.xlabel('€/MWh', size=20)
plt.ylabel('frequency', size=20)
plt.legend(prop={'size': 20})

plt.show()
basic_stats = sct.describe(original_savings,nan_policy='omit')
print('The total profit of the original strategy is: ',round(np.nansum(original_savings),2),' €, with a mean profit of: ',round(np.nanmean(original_savings),2),' €')
print('The total profit of the new strategy is: ',round(np.nansum(profit_LG),2),' €, with a mean profit of: ',round(np.nanmean(profit_LG),2),' €')
print('Improvement of: ',round(np.nansum(profit_LG),2)/round(np.nansum(original_savings),2)*100,' %')

In [ ]:
MC_profit.to_csv(r'C:\Users\luigi\OneDrive\2 University\4 ULB\0 THESIS\FINAL_SIMULATIONS\MC1.csv')

In [ ]:
pd.DataFrame(forecasts_LG).to_csv(r'C:\Users\luigi\OneDrive\2 University\4 ULB\0 THESIS\FINAL_SIMULATIONS\forecasts_OU.csv')
pd.DataFrame(final_prices_LG).to_csv(r'C:\Users\luigi\OneDrive\2 University\4 ULB\0 THESIS\FINAL_SIMULATIONS\final_prices_OU.csv')
pd.DataFrame(final_time_LG).to_csv(r'C:\Users\luigi\OneDrive\2 University\4 ULB\0 THESIS\FINAL_SIMULATIONS\final_time_OU.csv')
pd.DataFrame(profit_LG).to_csv(r'C:\Users\luigi\OneDrive\2 University\4 ULB\0 THESIS\FINAL_SIMULATIONS\profit_OU.csv')

In [ ]:
profit_ou = pd.read_csv(r'C:\Users\luigi\OneDrive\2 University\4 ULB\0 THESIS\FINAL_SIMULATIONS\profit_OU.csv')

In [ ]:
basic_stats = sct.describe(profit_ou,nan_policy='omit')
print(basic_stats)

In [ ]:
np.nansum(profit_ou)

In [ ]:
s_opt

In [ ]:
plt.figure(figsize = (20,10))
plt.plot(time,prediction,'r', label = 'Mean of the OU processes')
plt.plot(df.price[time],'b', label = 'Real series of prices')
plt.ylabel('€/MWh', size=20)
plt.title('Ornstein-Uhlenbeck forecast', size=30)
plt.legend(prop={'size': 30})
plt.show()

In [ ]:
len(prediction)

# 5 Trading strategy with Time Series

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
plt.rcParams.update({'figure.figsize':(9,7), 'figure.dpi':120})
from statsmodels.tsa.arima_model import ARIMA
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import quandl

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
plt.rcParams.update({'figure.figsize':(9,7), 'figure.dpi':120})
from statsmodels.tsa.arima_model import ARIMA
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import quandl

In [ ]:
import pmdarima as pm
from statsmodels.tsa.stattools import adfuller, kpss
from scipy import signal
from statsmodels.tsa.seasonal import seasonal_decompose

In [ ]:
df_trading

In [ ]:
i=0

In [ ]:
open_time = df_trading.open_position_time[i]
open_price = float(df_trading.price_at_open_time[i])

target_time = df_trading.target_time[i]
closing_price = float(df_trading.closing_price[i])
deadline = df_trading.datetime[i]

mask_train = (df.timestamp > (open_time - pd.Timedelta(value=15000, unit='min'))) & (df.timestamp < open_time)
df_train = df[mask_train]

In [ ]:
open_time

In [ ]:
plt.rcParams.update({'figure.figsize':(9,7), 'figure.dpi':120})
# Original Series

fig, axes = plt.subplots(3, 2)#, sharex=True)
axes[0, 0].plot(df_train.price.values); axes[0, 0].set_title('Original Series')
plot_acf(df_train.price, lags = 20, ax=axes[0, 1])

# 1st Differencing
axes[1, 0].plot(df_train.price.diff()); axes[1, 0]#.set_title('1st Order Differencing')
plot_acf(df_train.price.diff().dropna(), lags = 20, ax=axes[1, 1])

# log transformation
axes[2, 0].plot(np.log(df_train.price)); axes[2, 0]#.set_title('Log transformation')
plot_acf(np.log(df_train.price).dropna(), lags = 20, ax=axes[2, 1])

plt.show()

In [ ]:
plt.figure(figsize = (20,10))
#plt.plot(df_train.price)

transformation = np.log(df_train.price.diff())
transformation = transformation[transformation >-15]
plt.plot(transformation.diff(), label = 'difference of the log of the first difference of prices')
plt.plot(transformation, label = 'log of the first difference of prices',alpha = 0.7)
plt.legend()

In [ ]:
plt.rcParams.update({'figure.figsize':(9,7), 'figure.dpi':120})
# Original Series

fig, axes = plt.subplots(3, 2)#, sharex=True)
axes[0, 0].plot(df_train.price.diff().diff(240)); axes[0, 0].set_title('Original Series')
plot_acf(df_train.price.diff().diff(240), lags = 20, ax=axes[0, 1])

# 1st Differencing
axes[1, 0].plot(df_train.price.diff().diff(5760)); axes[1, 0]#.set_title('1st Order Differencing')
plot_acf(df_train.price.diff().diff(5760).dropna(), lags = 20, ax=axes[1, 1])

# log transformation
axes[2, 0].plot(df_train.price.diff().diff(40320)); axes[2, 0]#.set_title('Log transformation')
plot_acf(df_train.price.diff().diff(40320).dropna(), lags = 20, ax=axes[2, 1])

plt.show()

In [ ]:
i = 34
mask = df_trading.index == i


# We define some plotting parameters
selected_month = int(df_trading.open_position_time[mask].dt.month)
selected_day = int(df_trading.open_position_time[mask].dt.day)
selected_hour = int(df_trading.open_position_time[mask].dt.hour)
selected_minute = int(df_trading.open_position_time[mask].dt.minute)

selected_time = [selected_month, selected_day, selected_hour, selected_minute]

plot_mask = (df_prices.month == selected_month) & ((df_prices.day == selected_day) | (df_prices.day == (selected_day + 1)))


open_time = df_trading.open_position_time[mask][i]
open_price = float(df_trading.price_at_open_time[mask])

target_time = df_trading.target_time[mask][i]
closing_price = float(df_trading.closing_price[mask])
deadline = df_trading.datetime[mask][i]

In [ ]:
selected_hour = 0

features = df.price 

features_hour_0 = features[features.hour == 0]
train_features_0 = features_hour[(features_hour.month == 5) | ((features_hour.month == 6) & (features_hour.day <= 10)) ]

features_hour_1 = features[features.hour == 1]
train_features_1 = features_hour[(features_hour.month == 5) | ((features_hour.month == 6) & (features_hour.day <= 10)) ]

features_hour_1 = features[features.hour == 1]
train_features_1 = features_hour[(features_hour.month == 5) | ((features_hour.month == 6) & (features_hour.day <= 10)) ]

In [ ]:
fig, axes = plt.subplots(3, 2)#, sharex=True)
axes[0, 0].plot(train_features_0.price.diff()); axes[0, 0].set_title('Original Series')
plot_acf(train_features_0.price.diff().dropna(), lags = 20, ax=axes[0, 1])

# 1st Differencing
axes[1, 0].plot(train_features_1.price.diff()); axes[1, 0]#.set_title('1st Order Differencing')
plot_acf(train_features_1.price.diff().dropna(), lags = 20, ax=axes[1, 1])

# log transformation
axes[2, 0].plot(train_features_1.price.diff().diff()); axes[2, 0]#.set_title('Log transformation')
plot_acf(train_features_1.price.diff().diff().dropna(), lags = 20, ax=axes[2, 1])

plt.show()